In [1]:
!pip install -q streamlit transformers sentencepiece
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [14]:
%%writefile app.py
import os
import streamlit as st
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

st.set_page_config(page_title="Translation Bot", layout="centered")

MODEL_PATH = "saved_models/mbart50"
MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

@st.cache_resource
def load_model():
    if os.path.exists(MODEL_PATH):
        model = MBartForConditionalGeneration.from_pretrained(MODEL_PATH)
        tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_PATH)
    else:
        model = MBartForConditionalGeneration.from_pretrained(MODEL_NAME)
        tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_NAME)
        os.makedirs(MODEL_PATH, exist_ok=True)
        model.save_pretrained(MODEL_PATH)
        tokenizer.save_pretrained(MODEL_PATH)
    return model, tokenizer

model, tokenizer = load_model()

LANG_CODES = {
    "Arabic": "ar_AR",
    "English": "en_XX",
    "French": "fr_XX",
    "Turkish": "tr_TR",
    "Spanish": "es_XX",
    "German": "de_DE"
}

def translate(text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang]
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

st.title("🌍 Multilingual Translation Bot")

text = st.text_area("📝 Enter text to translate", "Welcome to the world of translation")

col1, col2 = st.columns(2)
with col1:
    src = st.selectbox("Choose source language", list(LANG_CODES.keys()), index=0)
with col2:
    tgt = st.selectbox("Choose target language", list(LANG_CODES.keys()), index=1)

if st.button("Translate"):
    if src == tgt:
        st.warning("🔁 You cannot translate to the same language!")
    else:
        translation = translate(text, LANG_CODES[src], LANG_CODES[tgt])
        st.success("✅ Translation:")
        st.write(translation)


Overwriting app.py


In [15]:
!ngrok config add-authtoken 2xGzTilikva2J6IqS4gsdbFeEHq_32ZJWr3i2mqwcXQHiBfB2


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
!rm -rf ~/.ngrok2


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501, bind_tls=True)
print("🔗 App URL:", public_url)

!streamlit run app.py &


🔗 App URL: NgrokTunnel: "https://8008-34-106-203-96.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.106.203.96:8502

